In [14]:
import numpy as np
from pynput import keyboard
import pylink
import pyperclip
import random
import autopy.mouse as mouse
import autopy.key as key
import win32api, win32con
import sys, os, time
pylink.__version__

'1.11.0.0'

In [18]:
sub_no = input("Please input subject number：")
sub_gender = input("Please input subject gender(male：m，female：f)：")
sub_age = input("Please input subject age：")

print('Subject', sub_no, "recorded")

Please input subject number： 10
Please input subject gender(male：m，female：f)： m
Please input subject age： 24


Subject 10 recorded


In [19]:
np.seterr(divide='ignore',invalid='ignore')
class EyeMouse():
    ''' using the eyes to control the mouse on Windows machine'''
    
    def __init__(self, eyelinkTracker):
        self.tracker = eyelinkTracker
        self.pos = mouse.location()
        self.pos_series = np.array([])
        self.win_len = 8
        self.threshold = [0, 200]
        self.temp = []
        
    def gaze_filter(self, new):
        if len(self.pos_series) == 0:
            new.append(1)
            self.temp = new[:-1]
            self.pos_series = np.append(self.pos_series, new).reshape(1,3)
            return [new[0], new[1]]
        else:
            E = np.sum(self.pos_series[-self.win_len:, -1])
            p = [np.sum(np.multiply(self.pos_series[-self.win_len:, 0], self.pos_series[-self.win_len:, -1])),
                 np.sum(np.multiply(self.pos_series[-self.win_len:, 1], self.pos_series[-self.win_len:, -1]))]
            pm = [p[0] / E, p[1] / E]
            
            distence = np.linalg.norm(np.array(new)-np.array(pm))

            if np.isnan(distence):
                new.append(1)
            elif self.threshold[0] < int(distence) < self.threshold[1]:    #threshold
                new.append(1)
            else:
                new.append(0)
                
            if new[0] < 0:
                new[0] = 0
            if new[1] < 0:
                new[1] = 0
                
            self.pos_series = np.append(self.pos_series, new).reshape(len(self.pos_series) + 1,3)
            
            pi_E = np.sum(self.pos_series[-self.win_len:, -1])
            global pi
            pi = [(np.sum(np.multiply(self.pos_series[-self.win_len:-1, 0], self.pos_series[-self.win_len:-1, -1])) + new[0] * new[-1]) / pi_E,
                  (np.sum(np.multiply(self.pos_series[-self.win_len:-1, 1], self.pos_series[-self.win_len:-1, -1])) + new[1] * new[-1]) / pi_E]
            
            if len(self.pos_series) >= 21:
                self.pos_series = np.delete(self.pos_series, 0, axis = 0)
                
            if np.linalg.norm(np.array(new[:-1])-np.array(pi)) > 80:
                self.temp = pi
                return pi
            else:
                return self.temp

    def updateMouse(self):
        ''' update the mouse postion'''
        #if self.tracker.eyeAvailable()>0: # 0-Left,1-Right,2-Binocular, -1-No eye
        newEvent = self.tracker.getNextData()
        tmpData = self.tracker.getFloatData()

        if newEvent >0:
            # use the blink events to change the mouse states, blink = button down, blink end = button up

            if tmpData and newEvent == pylink.FIXUPDATE:
                self.pos = self.gaze_filter(list(map(int, tmpData.getAverageGaze())))
                if self.pos[0] > 0 and self.pos[0] < 1920 and self.pos[1] > 0 and self.pos[1] < 1080:
                    if self.temp == pi:
                        mouse.move(self.pos[0], self.pos[1])

def open_page(url):
    mouse.move(267, 19)
    mouse.click()
    mouse.move(600, 50)
    mouse.click()
    key.type_string(url)
    win32api.keybd_event(0x0D,0,0,0) 
    mouse.move(960, 540)
    pass

def close_page(repeat):
    for i in range(repeat):
        mouse.move(470, 15)
        time.sleep(1)
        mouse.click()
    pass


def on_release(key):
    if key == keyboard.Key.insert:
        return False
    
def runTrial():
    '''校正程序'''
    tk.doTrackerSetup()

    # 强制离线模式防止视线漂移 
    pylink.flushGetkeyQueue() # clear all cached button responses

    tk.setOfflineMode()
    pylink.pumpDelay(100);
    pylink.closeGraphics()
    
    tk.sendMessage("SYNCTIME %d" % pylink.currentTime());
    my_mouse = EyeMouse(tk)
    urls = ['www.amazon.cn', 'www.tmall.com', 'www.jd.com']
    
    
#===========================test_trial===========================

    lapse = []
    pylink.openGraphics()
    tk.doTrackerSetup()

    # 强制离线模式防止视线漂移 
    pylink.flushGetkeyQueue() # clear all cached button responses

    tk.setOfflineMode()
    pylink.pumpDelay(100);
    pylink.closeGraphics()
    with open('listen.txt', 'w') as f:
        f.close()
    pyperclip.copy("苹果手机")
    '''检测开始按键'''
    with keyboard.Listener(on_release=on_release) as listener:
        listener.join()
    
    tk.startRecording(1,1,1,1)
    tk.sendMessage("TEST_TRAIL_START")
    open_page(urls[0])
    startTime = pylink.currentTime()
    while True:
        my_mouse.updateMouse()
        lapse.append(pylink.currentTime() - startTime)
        if len(lapse) > 80:
            lapse.pop(0)
        if lapse[-1] > 3000 and lapse[-1] % 1000 == 0:
            with open('listen.txt', 'r') as f:
                if f.read() == 'esc':
                    tk.sendMessage("TEST_TRAIL_END")
                    tk.stopRecording()
                    '''检测开始按键'''
                    with keyboard.Listener(on_release=on_release) as listener:
                        listener.join()
                    break      
    time.sleep(2)
    close_page(4)
        
        
#===========================record_trial===========================   
    random.shuffle(order)
    
        
    for i in range(2):
        pylink.openGraphics()
        tk.doTrackerSetup()

        # 强制离线模式防止视线漂移 
        pylink.flushGetkeyQueue() # clear all cached button responses

        tk.setOfflineMode()
        pylink.pumpDelay(100);
        pylink.closeGraphics()
        with open('listen.txt', 'w') as f:
            f.close()
        pyperclip.copy("华为手机")
        '''检测开始按键'''
        with keyboard.Listener(on_release=on_release) as listener:
            listener.join()
        tk.startRecording(1,1,1,1)
        tk.sendMessage("Record_" + urls[order[i]][4:-4] + "_Start_at_" + time.asctime(time.localtime())[-13:-5])
        lapse = []
        open_page(urls[order[i]])
        startTime = pylink.currentTime()
        while True:
            my_mouse.updateMouse()
            lapse.append(pylink.currentTime() - startTime)
            if len(lapse) > 80:
                lapse.pop(0)
            if lapse[-1] > 3000 and lapse[-1] % 1000 == 0:
                with open('listen.txt', 'r') as f:
                    if f.read() == 'esc':
                        tk.sendMessage("Record_" + urls[order[i]][4:-4] + "_End")
                        tk.stopRecording()
                        '''检测开始按键'''
                        with keyboard.Listener(on_release=on_release) as listener:
                            listener.join()
                        break
        time.sleep(2)
        close_page(4)
        with open('listen.txt', 'w') as f:
            f.close()
        
        
##    tk.sendMessage('TRIAL OK')
    pylink.pumpDelay(100)
    tk.stopRecording()
    tk.receiveDataFile(dataFileName, dataFolder + dataFileName)
    tk.close()

In [20]:
# 步骤一：连接眼动仪
dummyMode = False 
if not dummyMode: 
    tk = pylink.EyeLink('100.1.1.1')
else:
    tk = pylink.EyeLink(None)

#### STEP II: 创建EDF文件 ####################################################

dataFolder = os.getcwd() + '/data/edf/'

if not os.path.exists(dataFolder): 
    os.makedirs(dataFolder)

dataFileName = sub_no + '_' + sub_gender + '_' + sub_age + '.EDF'

# Note that for Eyelink 1000/II, the file name cannot exceeds 8 characters
# we need to open eyelink data files early so as to record as much info as possible
tk.openDataFile(dataFileName)
disp = pylink.getDisplayInformation()
print(disp.width, disp.height, disp.bits, disp.refresh)
pylink.openGraphics()

# 步骤四：使用'sendCommand'方法设置眼动仪参数
tk.sendCommand('sample_rate 250')
# 设定校准屏幕的主题颜色
pylink.setCalibrationColors((255,255,255), (0,0,0))

#### Zhiguo: These paramters are for specifying what data will be stored and can be accessed online
#### The file_event_filter etc are unnecessary for your purposes, I believe
#### The link_sample_data link_event_filter parameters are vital
# 设定在线分析程序
# 0--> 标准 (认知实验); 1--> 敏感 (神经心理学)
tk.sendCommand('select_parser_configuration 1')
# 设置EDF文件存储内容（事件相关：左、右、注视、扫视、眨眼、消息、按键）
tk.sendCommand("file_event_filter = LEFT,RIGHT,FIXATION,SACCADE,BLINK,MESSAGE,BUTTON")
tk.sendCommand("file_sample_data  = LEFT,RIGHT,GAZE,AREA,GAZERES,STATUS,HTARGET")  

# 设置连接参数（用于视线追随）
tk.sendCommand("link_event_filter = LEFT,RIGHT,FIXATION,SACCADE,BLINK,FIXUPDATE,MESSAGE,BUTTON")
tk.sendCommand("link_sample_data  = LEFT,RIGHT,GAZE,GAZERES,AREA,STATUS,HTARGET")
#tk.setLinkSampleData("LEFT,RIGHT,GAZE,GAZERES,AREA,STATUS,HTARGET") # you can also use the setXXX comand

#### 设定九点校准
tk.sendCommand("calibration_type = HV9")

#### 告知眼动仪屏幕数据
tk.sendCommand("screen_pixel_coords = 0 0 %d %d" % (disp.width-1, disp.height-1))

#### 双眼模式
# tk.sendCommand("binocular_enabled = YES")
tk.sendCommand("pupil_size_diameter = YES")

#### 设置注视更新
# tk.sendCommand("fixation_update_interval = 80")
# tk.sendCommand("fixation_update_accumulate = 80")
# tk.sendCommand("heuristic_filter = ON")

# #### 设置Event阈限
# tk.sendCommand("saccade_velocity_threshold = 200")
# tk.sendCommand("saccade_acceleration_threshold = 20000")
# tk.sendCommand("saccade_motion_threshold = 0.8")
runTrial()
pylink.pumpDelay(100) # ISI of 1000 ms

1920 1080 32 59.0
